In [1]:
# Dependencies
import requests
import json
import pprint
from pymongo import MongoClient


In [2]:
# request to get JSON that contains all team ids
teams_response = requests.get('https://statsapi.web.nhl.com/api/v1/teams').json()

In [3]:
# pretty print teams_response
pp = pprint.PrettyPrinter(indent=4)
pp.pprint(teams_response)

{   'copyright': 'NHL and the NHL Shield are registered trademarks of the '
                 'National Hockey League. NHL and NHL team marks are the '
                 'property of the NHL and its teams. © NHL 2018. All Rights '
                 'Reserved.',
    'teams': [   {   'abbreviation': 'NJD',
                     'active': True,
                     'conference': {   'id': 6,
                                       'link': '/api/v1/conferences/6',
                                       'name': 'Eastern'},
                     'division': {   'abbreviation': 'M',
                                     'id': 18,
                                     'link': '/api/v1/divisions/18',
                                     'name': 'Metropolitan',
                                     'nameShort': 'Metro'},
                     'firstYearOfPlay': '1982',
                     'franchise': {   'franchiseId': 23,
                                      'link': '/api/v1/franchises/23',
          

                                     'name': 'Atlantic',
                                     'nameShort': 'ATL'},
                     'firstYearOfPlay': '1990',
                     'franchise': {   'franchiseId': 30,
                                      'link': '/api/v1/franchises/30',
                                      'teamName': 'Senators'},
                     'franchiseId': 30,
                     'id': 9,
                     'link': '/api/v1/teams/9',
                     'locationName': 'Ottawa',
                     'name': 'Ottawa Senators',
                     'officialSiteUrl': 'http://www.ottawasenators.com/',
                     'shortName': 'Ottawa',
                     'teamName': 'Senators',
                     'venue': {   'city': 'Ottawa',
                                  'id': 5031,
                                  'link': '/api/v1/venues/5031',
                                  'name': 'Canadian Tire Centre',
                                  'timeZo

In [4]:
# Number of teams in the NHL
len(teams_response['teams'])

31

In [5]:
# Collects the ids of active NHL teams

team_ids = []

for x in range(len(teams_response['teams'])):
    team_ids.append(teams_response['teams'][x]['id'])

In [6]:
# Uses the collected ids to loop through, pull stats & write to MongoDB

for team in team_ids:
    
    query_url = f"https://statsapi.web.nhl.com/api/v1/teams/{team}/stats"
    team_stats = requests.get(query_url).json()
    
    client = MongoClient('localhost', 27017)
    db = client['nhl-database']
    collection = db['STATS']
    collection.insert_one(team_stats)
    

In [7]:
import pandas as pd
from pandas.io.json import json_normalize

In [8]:
datapoints = pd.DataFrame(list(collection.find({})))

In [9]:
pre_df_data = []

for i in range(len(team_ids)):
    pre_df_data.append((datapoints.stats[i][0]['splits'][0]))

In [10]:
stats_df = json_normalize(pre_df_data)
stats_df

,stat.evGGARatio,stat.faceOffWinPercentage,stat.faceOffsLost,stat.faceOffsTaken,stat.faceOffsWon,stat.gamesPlayed,stat.goalsAgainstPerGame,stat.goalsPerGame,stat.losses,stat.ot,...,stat.winLeadFirstPer,stat.winLeadSecondPer,stat.winOppScoreFirst,stat.winOutshootOpp,stat.winOutshotByOpp,stat.winScoreFirst,stat.wins,team.id,team.link,team.name
0,0.8929,46.7,885.0,1661.0,776.0,26,3.500,2.885,12,5,...,0.600,0.800,0.100,0.400,0.375,0.500,9,1,/api/v1/teams/1,New Jersey Devils
1,1.2500,47.5,771.0,1468.0,697.0,26,2.808,2.923,10,3,...,0.545,0.846,0.400,0.125,0.706,0.563,13,2,/api/v1/teams/2,New York Islanders
2,0.8750,48.7,819.0,1596.0,777.0,28,3.107,2.714,12,3,...,0.400,0.714,0.462,0.500,0.444,0.467,13,3,/api/v1/teams/3,New York Rangers
3,0.9455,55.9,631.0,1430.0,799.0,25,3.520,3.000,12,2,...,0.778,0.818,0.313,0.267,0.571,0.667,11,4,/api/v1/teams/4,Philadelphia Flyers
4,1.1356,49.9,816.0,1630.0,814.0,26,3.269,3.423,10,5,...,0.545,0.900,0.333,0.364,0.500,0.471,11,5,/api/v1/teams/5,Pittsburgh Penguins
5,1.1667,49.0,832.0,1633.0,801.0,27,2.556,2.593,9,4,...,0.818,0.909,0.333,0.455,0.533,0.750,14,6,/api/v1/teams/6,Boston Bruins
6,1.1154,47.1,932.0,1763.0,831.0,29,2.828,2.966,8,4,...,0.833,0.800,0.357,0.500,0.625,0.800,17,7,/api/v1/teams/7,Buffalo Sabres
7,1.1167,47.0,920.0,1737.0,817.0,28,3.214,3.071,10,5,...,0.778,0.700,0.308,0.500,0.417,0.600,13,8,/api/v1/teams/8,Montréal Canadiens
8,0.8961,50.2,888.0,1785.0,897.0,28,4.071,3.571,13,3,...,0.583,0.700,0.357,0.625,0.350,0.500,12,9,/api/v1/teams/9,Ottawa Senators
9,1.3878,51.7,834.0,1728.0,894.0,28,2.607,3.643,8,0,...,1.000,1.000,0.429,0.636,0.813,1.000,20,10,/api/v1/teams/10,Toronto Maple Leafs


In [11]:
teams_array = stats_df['team.name'].values
games_played_array = stats_df['stat.gamesPlayed'].values
wins_array = stats_df['stat.wins'].values
losses_array = stats_df['stat.losses'].values
fo_win_array = stats_df['stat.faceOffWinPercentage'].values
gpg_array = stats_df['stat.goalsPerGame'].values
gapg_array = stats_df['stat.goalsAgainstPerGame'].values
win_outshootpct_array = stats_df['stat.winOutshootOpp'].values
win_scorefirstpct_array = stats_df['stat.winScoreFirst'].values

In [12]:
win_scorefirstpct_array

array([0.5  , 0.563, 0.467, 0.667, 0.471, 0.75 , 0.8  , 0.6  , 0.5  ,
       1.   , 0.75 , 0.385, 0.8  , 0.727, 0.583, 0.5  , 0.938, 0.538,
       0.857, 0.733, 0.615, 0.5  , 0.692, 0.647, 0.818, 0.667, 0.643,
       0.556, 0.533, 0.786, 0.625])

In [13]:
from flask import Flask, render_template, jsonify
from flask_pymongo import PyMongo

app = Flask(__name__)
app.config["MONGO_URI"] = "mongodb://localhost:27017/nhl-database"
mongo = PyMongo(app)

stats = []
team_stats = list(mongo.db.STATS.find({}, {'_id': False}))
for stat in team_stats:
    stats.append({
            "AAA" : stat["stats"]
            })

In [15]:
stats_df.loc[:, 'stat.faceOffWinPercentage']

0     46.7
1     47.5
2     48.7
3     55.9
4     49.9
5     49.0
6     47.1
7     47.0
8     50.2
9     51.7
10    51.8
11    49.8
12    50.4
13    48.6
14    49.7
15    52.5
16    52.3
17    53.1
18    51.8
19    46.6
20    47.7
21    47.9
22    50.4
23    51.9
24    52.8
25    49.4
26    48.6
27    52.0
28    51.8
29    49.1
30    48.6
Name: stat.faceOffWinPercentage, dtype: object

In [29]:
stats_df.to_dict

<bound method DataFrame.to_dict of     stat.evGGARatio stat.faceOffWinPercentage  stat.faceOffsLost  \
0            0.8929                      46.7              885.0   
1            1.2500                      47.5              771.0   
2            0.8750                      48.7              819.0   
3            0.9455                      55.9              631.0   
4            1.1356                      49.9              816.0   
5            1.1667                      49.0              832.0   
6            1.1154                      47.1              932.0   
7            1.1167                      47.0              920.0   
8            0.8961                      50.2              888.0   
9            1.3878                      51.7              834.0   
10           0.9333                      51.8              771.0   
11           0.8750                      49.8              855.0   
12           1.2373                      50.4              864.0   
13           